<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/139_B2B_Sales_Agent_Claude_04_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The way you’ve structured things (with `research_agent.py`, `analysis_agent.py`, `personalization_agent.py`, and then `sales_orchestrator.py` pulling them together) **is very much best practice** in Python projects .

Here’s why this separation into individual `.py` files (modules) is the clean approach:

---

## 🔹 Why separate agents into their own `.py` files?

1. **Single Responsibility Principle (SRP)**
   Each agent has one purpose:

   * `research_agent.py` → gather company info.
   * `analysis_agent.py` → analyze pain points/opportunities.
   * `personalization_agent.py` → generate outreach.
     Keeping each in its own file avoids bloated “god classes” and keeps logic focused.

2. **Reusability**
   If you wanted to reuse the `AnalysisAgent` in another project (say, not sales, but market analysis), you can import it without dragging in the whole sales pipeline.

3. **Testability**
   You can unit test each agent in isolation. For example:

   ```bash
   pytest tests/test_analysis_agent.py
   ```

   Much easier than testing a giant single file.

4. **Scalability**
   As your system grows (more agents, more steps), adding a new file is cleaner than stuffing everything into one mega-module.

5. **Orchestrator stays lean**
   In `sales_orchestrator.py`, you just import the agents:

   ```python
   from research_agent import ResearchAgent, CompanyInfo
   from analysis_agent import AnalysisAgent, AnalysisResult
   from personalization_agent import PersonalizationAgent, PersonalizationResult
   ```

   This makes the orchestrator focus only on *workflow*, not agent internals .

---

## 🔹 Where to store data/config (like industry patterns)

For static knowledge like your `_industry_patterns`, best practice is:

* Keep **logic in `.py` files**.
* Keep **data/config in JSON/YAML** files.

That way:

* Agents remain small, logic-focused modules.
* Updating patterns is a *data-editing* task, not a *code-editing* task.
* Orchestrator/agents just `load` what they need.

---

## 🔹 Typical project layout

Here’s what a clean structure might look like:

```
sales_agents/
│
├── agents/
│   ├── research_agent.py
│   ├── analysis_agent.py
│   ├── personalization_agent.py
│
├── config/
│   ├── industry_patterns.json
│   └── icp_rules.json
│
├── orchestrator/
│   └── sales_orchestrator.py
│
├── tests/
│   ├── test_research_agent.py
│   ├── test_analysis_agent.py
│   └── test_personalization_agent.py
│
└── main.py   # entry point
```

* `agents/` → reusable building blocks.
* `config/` → knowledge & ICP definitions.
* `orchestrator/` → workflow logic.
* `tests/` → unit/integration tests.
* `main.py` → runs the pipeline.

---

✅ So yes — **separating each agent into its own `.py` file is best practice**. Then the orchestrator (or any higher-level script) just references them and coordinates.




# Pipeline Code

In [ ]:
"""
Demo Script - Complete Sales Pipeline Orchestration

This script demonstrates:
- Individual agent testing
- Complete pipeline execution
- Error handling and recovery
- Performance monitoring
- Results visualization
"""

import json
import time
from datetime import datetime
from sales_orchestrator import SalesOrchestrator, WorkflowStatus
from research_agent import ResearchAgent
from analysis_agent import AnalysisAgent
from personalization_agent import PersonalizationAgent

def print_separator(title: str = ""):
    """Print a formatted separator"""
    if title:
        print(f"\n{'='*60}")
        print(f"  {title}")
        print(f"{'='*60}")
    else:
        print(f"\n{'='*60}")

def print_step_result(step_name: str, success: bool, details: str = ""):
    """Print formatted step result"""
    status = "✅ SUCCESS" if success else "❌ FAILED"
    print(f"{status} - {step_name}")
    if details:
        print(f"    {details}")

def demo_individual_agents():
    """Demo individual agents working in isolation"""
    print_separator("INDIVIDUAL AGENT DEMONSTRATION")

    # Test Research Agent
    print("\n1. Testing Research Agent...")
    research_agent = ResearchAgent()

    companies_to_test = ["Acme Corporation", "TechStartup Inc", "Unknown Company"]

    for company in companies_to_test:
        try:
            company_info = research_agent.research_company(company)
            if company_info:
                print_step_result(f"Research: {company}", True,
                               f"Found {company_info.industry} company with {len(company_info.key_contacts)} contacts")
            else:
                print_step_result(f"Research: {company}", False, "No information found")
        except Exception as e:
            print_step_result(f"Research: {company}", False, str(e))

    # Test Analysis Agent
    print("\n2. Testing Analysis Agent...")
    analysis_agent = AnalysisAgent()

    # Get a known company for analysis
    company_info = research_agent.research_company("Acme Corporation")
    if company_info:
        try:
            analysis_result = analysis_agent.analyze_company(company_info)
            print_step_result("Analysis: Acme Corporation", True,
                           f"Found {len(analysis_result.pain_points)} pain points, "
                           f"{len(analysis_result.opportunities)} opportunities, "
                           f"confidence: {analysis_result.confidence_score:.2f}")
        except Exception as e:
            print_step_result("Analysis: Acme Corporation", False, str(e))

    # Test Personalization Agent
    print("\n3. Testing Personalization Agent...")
    personalization_agent = PersonalizationAgent()

    if company_info and 'analysis_result' in locals():
        try:
            personalization_result = personalization_agent.personalize_outreach(
                company_info, analysis_result, "Demo User"
            )
            print_step_result("Personalization: Acme Corporation", True,
                           f"Created {len(personalization_result.messages)} messages, "
                           f"strategy: {personalization_result.personalization_strategy}")
        except Exception as e:
            print_step_result("Personalization: Acme Corporation", False, str(e))

def demo_complete_pipeline():
    """Demo the complete orchestrated pipeline"""
    print_separator("COMPLETE PIPELINE DEMONSTRATION")

    # Create orchestrator
    orchestrator = SalesOrchestrator()

    # Test companies
    test_companies = [
        {"name": "Acme Corporation", "sender": "John Smith"},
        {"name": "TechStartup Inc", "sender": "Sarah Johnson"},
        {"name": "Unknown Company", "sender": "Mike Chen"}  # This should fail gracefully
    ]

    results = []

    for test_case in test_companies:
        print(f"\n🚀 Executing pipeline for: {test_case['name']}")
        print(f"   Sender: {test_case['sender']}")

        start_time = time.time()

        try:
            # Execute the complete pipeline
            workflow_state = orchestrator.execute_sales_pipeline(
                test_case['name'],
                test_case['sender']
            )

            execution_time = time.time() - start_time

            # Analyze results
            success = workflow_state.status == WorkflowStatus.COMPLETED
            print_step_result(f"Pipeline: {test_case['name']}", success,
                           f"Execution time: {execution_time:.2f}s")

            if success:
                # Show detailed results
                personalization_result = workflow_state.steps[2].output_data["personalization_result"]

                print(f"   📊 Results:")
                print(f"      Strategy: {personalization_result.personalization_strategy}")
                print(f"      Messages: {len(personalization_result.messages)}")
                print(f"      Sequence: {', '.join(personalization_result.recommended_sequence)}")

                # Show sample message
                email_msg = next((msg for msg in personalization_result.messages if msg.channel == "email"), None)
                if email_msg:
                    print(f"      Sample Subject: {email_msg.subject}")

                results.append({
                    "company": test_case['name'],
                    "success": True,
                    "execution_time": execution_time,
                    "strategy": personalization_result.personalization_strategy,
                    "messages_count": len(personalization_result.messages)
                })
            else:
                print(f"   ❌ Failed: {workflow_state.error_message}")
                results.append({
                    "company": test_case['name'],
                    "success": False,
                    "execution_time": execution_time,
                    "error": workflow_state.error_message
                })

        except Exception as e:
            execution_time = time.time() - start_time
            print_step_result(f"Pipeline: {test_case['name']}", False, str(e))
            results.append({
                "company": test_case['name'],
                "success": False,
                "execution_time": execution_time,
                "error": str(e)
            })

    return results

def demo_error_handling():
    """Demo error handling and recovery"""
    print_separator("ERROR HANDLING DEMONSTRATION")

    orchestrator = SalesOrchestrator()

    # Test with a company that will fail
    print("\n🔧 Testing error handling with invalid input...")

    try:
        workflow_state = orchestrator.execute_sales_pipeline("", "Test User")  # Empty company name
        print_step_result("Empty company name", False, "Should have failed validation")
    except Exception as e:
        print_step_result("Empty company name", True, f"Properly caught error: {str(e)}")

    # Test workflow status monitoring
    print("\n📊 Testing workflow monitoring...")
    workflows = orchestrator.get_all_workflows()
    print(f"   Active workflows: {len(workflows)}")

    for workflow_id, workflow_state in workflows.items():
        print(f"   Workflow {workflow_id}: {workflow_state.status.value}")

def demo_performance_metrics():
    """Demo performance monitoring and metrics"""
    print_separator("PERFORMANCE METRICS")

    orchestrator = SalesOrchestrator()

    # Get orchestrator status
    status = orchestrator.get_orchestrator_status()

    print(f"\n📈 Orchestrator Metrics:")
    print(f"   Total Workflows: {status['total_workflows']}")
    print(f"   Completed: {status['completed_workflows']}")
    print(f"   Failed: {status['failed_workflows']}")
    print(f"   Success Rate: {status['success_rate']:.1%}")
    print(f"   Workflow Steps: {status['workflow_steps']}")

    print(f"\n🤖 Agent Status:")
    for agent_status in status['active_agents']:
        print(f"   {agent_status['agent_id']}: {agent_status['status']}")

def demo_message_samples():
    """Demo sample personalized messages"""
    print_separator("SAMPLE PERSONALIZED MESSAGES")

    orchestrator = SalesOrchestrator()

    # Execute pipeline for a known company
    workflow_state = orchestrator.execute_sales_pipeline("Acme Corporation", "Alex Rodriguez")

    if workflow_state.status == WorkflowStatus.COMPLETED:
        personalization_result = workflow_state.steps[2].output_data["personalization_result"]

        print(f"\n📧 Generated Messages for {personalization_result.company_name}:")
        print(f"   Strategy: {personalization_result.personalization_strategy}")
        print(f"   Primary Contact: {personalization_result.primary_contact}")

        for i, message in enumerate(personalization_result.messages, 1):
            print(f"\n   {i}. {message.channel.upper()} Message:")
            print(f"      Subject: {message.subject}")
            print(f"      Tone: {message.tone}")
            print(f"      CTA: {message.call_to_action}")
            print(f"      Personalized Elements:")
            for element in message.personalization_elements:
                print(f"         • {element}")
            print(f"      Body Preview:")
            print(f"         {message.body[:300]}...")

def main():
    """Main demo function"""
    print_separator("AI AGENT ORCHESTRATION DEMO")
    print("This demo shows how individual agents work together")
    print("through orchestration to create a complete sales pipeline.")

    # Run all demonstrations
    demo_individual_agents()

    pipeline_results = demo_complete_pipeline()

    demo_error_handling()

    demo_performance_metrics()

    demo_message_samples()

    # Summary
    print_separator("DEMO SUMMARY")

    successful_pipelines = [r for r in pipeline_results if r['success']]
    failed_pipelines = [r for r in pipeline_results if not r['success']]

    print(f"\n📊 Pipeline Results:")
    print(f"   Successful: {len(successful_pipelines)}")
    print(f"   Failed: {len(failed_pipelines)}")
    print(f"   Success Rate: {len(successful_pipelines)/len(pipeline_results):.1%}")

    if successful_pipelines:
        avg_time = sum(r['execution_time'] for r in successful_pipelines) / len(successful_pipelines)
        print(f"   Average Execution Time: {avg_time:.2f}s")

    print(f"\n🎯 Key Learnings:")
    print(f"   • Orchestration manages complex multi-agent workflows")
    print(f"   • Error handling ensures graceful failure recovery")
    print(f"   • State management tracks data flow between agents")
    print(f"   • Monitoring provides visibility into system performance")
    print(f"   • Personalization creates targeted, relevant outreach")

    print(f"\n🚀 Next Steps:")
    print(f"   • Integrate with real APIs (web scraping, CRM systems)")
    print(f"   • Add LLM integration for natural language generation")
    print(f"   • Implement A/B testing for message optimization")
    print(f"   • Add human-in-the-loop approval workflows")
    print(f"   • Scale to handle multiple concurrent pipelines")

if __name__ == "__main__":
    main()


## Pipeline Code Explained
This `demo_sales_pipeline.py` is the *capstone script* that ties together all the pieces you’ve been building (research agent, analysis agent, personalization agent, and the orchestrator). Let’s break it down.

---

# 🔎 What This Pipeline Demo Is Doing

### 1. **Demonstrates agents individually**

* It first runs the **ResearchAgent**, **AnalysisAgent**, and **PersonalizationAgent** in isolation.
* This lets you test each module before chaining them together.
* Example: check if ResearchAgent can fetch company info and return structured contacts.

---

### 2. **Runs the complete orchestrated pipeline**

* Creates a `SalesOrchestrator`.
* Runs the full workflow for different companies (`Acme Corporation`, `TechStartup Inc`, and an unknown one to simulate failure).
* Measures execution time, prints results, and captures strategies/messages generated.
* The orchestrator sequences the agents, tracks workflow state, and enforces status transitions.

---

### 3. **Error handling and recovery**

* Deliberately passes a bad input (empty company name) to show how the orchestrator catches errors.
* Demonstrates graceful failure handling and logs the failure rather than crashing.

---

### 4. **Performance monitoring**

* Shows metrics like total workflows, success rate, failed workflows, and agent statuses.
* This is crucial when you scale — monitoring tells you if pipelines are healthy, fast, and reliable.

---

### 5. **Outputs sample messages**

* Runs the full pipeline on a known company and prints the *actual personalized outreach messages* generated.
* This lets you inspect tone, subject lines, CTAs, etc. for realism.

---

### 6. **Summarizes key learnings**

At the end, it prints a summary with:

* Success vs failed runs
* Average execution time
* Key principles: orchestration, error handling, state management, monitoring, personalization.

---

# ✨ Key Takeaways You Should Be Learning

1. **End-to-End Orchestration**

   * How individual agents (research, analysis, personalization) can be chained into a larger workflow with an orchestrator.
   * This is how you scale from “toy agents” to *production pipelines*.

2. **State & Status Management**

   * Each workflow has statuses (`PENDING`, `COMPLETED`, `FAILED`) and each step has tracked inputs/outputs.
   * You learn how to avoid “black box” LLM runs by keeping structured state.

3. **Error Handling is First-Class**

   * Instead of crashing, errors are logged, workflows fail gracefully, and retries are possible.
   * This is production-grade design: assume some steps *will* fail.

4. **Observability & Metrics**

   * Monitoring success rate, execution time, and active agent status helps you *trust and debug* the system.

5. **Personalization in Action**

   * The final step demonstrates how structured research → analysis → outreach can produce targeted, useful outputs.

---

# 🚀 What To Focus On As You Learn

* **Workflow design** → how multiple agents fit together.
* **Interfaces/contracts** → every agent returns structured objects, making orchestration predictable.
* **Failure resilience** → don’t just run things, manage what happens when they fail.
* **Observability** → logs, metrics, summaries are just as important as outputs.
* **Incremental testing** → the script shows a good pattern: test agents individually, then as a pipeline.

---

👉 In short: this script isn’t about adding *new capabilities*, it’s about showing you how to **run, monitor, and trust a complete agent pipeline** — the last step before integrating with real APIs and scaling to production.




# 🔹 Orchestrator vs Pipeline

### **1. The Orchestrator Agent**

* Think of it as the **conductor** of an orchestra.
* It **owns and manages the agents** (ResearchAgent, AnalysisAgent, PersonalizationAgent).
* Responsibilities:

  * Decides which agent runs next.
  * Tracks state & status of each step.
  * Handles retries, errors, and human-in-the-loop breaks.
  * Maintains the workflow log.
* On its own, it’s just the *control system* — it doesn’t *run* anything unless told to.

---

### **2. The Pipeline Script**

* This is the **demo runner / execution environment**.
* It imports the orchestrator, **configures** it, and tells it:

  * “Run the workflow for Acme Corp.”
  * “Now run it again for TechStartup.”
  * “Now simulate an error with a bad input.”
* It **wraps the orchestrator in a runnable script** with monitoring, logging, and example outputs.

---

# 🔹 How They Fit Together

Here’s the hierarchy:

```
Pipeline (demo_sales_pipeline.py)
   |
   |---> Creates an instance of SalesOrchestrator
               |
               |---> Inside: runs ResearchAgent
               |---> then AnalysisAgent
               |---> then PersonalizationAgent
               |---> tracks state, retries, errors
```

* **Pipeline = Flight plan (what flights to run, when, and how to monitor results).**
* **Orchestrator = Air traffic control tower (decides which plane takes off next, checks weather, handles emergencies).**
* **Agents = Airplanes (each with a specific job to do).**

---

# 🔹 Why Both Exist

* If you only had agents → you’d just have disconnected workers, no coordination.
* If you only had the orchestrator → you’d have a conductor, but no “concert to perform.”
* If you only had a pipeline → it would be a rigid script, not adaptive.

By combining them:

* **Pipeline gives context + execution scope.**
* **Orchestrator manages the actual workflow logic.**
* **Agents perform the domain-specific tasks.**

---

✅ **Key learning:**

* The **orchestrator is embedded *inside* the pipeline**.
* The pipeline sets up *when/how to run workflows*.
* The orchestrator makes sure *the workflows run correctly*.





### 🏗️ **Architecture Overview**

**Individual Agents:**
1. **Research Agent** - Finds company information from web sources
2. **Analysis Agent** - Identifies pain points and opportunities
3. **Personalization Agent** - Creates custom outreach messages

**Orchestrator:**
- **Sales Orchestrator** - Manages workflow, state, error handling, and agent coordination

### 🔧 **Key Software Engineering Concepts Demonstrated**

**1. Clean Architecture**
- Separation of concerns (each agent has a single responsibility)
- Dependency injection and modular design
- Type hints and dataclasses for data structures

**2. Error Handling & Resilience**
- Graceful failure handling at each step
- Retry logic and error propagation
- Comprehensive logging and monitoring

**3. State Management**
- Workflow state tracking across multiple agents
- Data flow between agents (Research → Analysis → Personalization)
- Status monitoring and metrics

**4. Orchestration Patterns**
- Sequential workflow execution
- Agent coordination and handoffs
- Human-in-the-loop capabilities (ready for extension)

### 🎯 **What Makes This Different from Typical Agents**

**Typical Agent:**
```python
# Single-purpose agent
def simple_agent(input):
    return process(input)
```

**Our Orchestrated System:**
```python
# Multi-agent workflow with state management
orchestrator.execute_sales_pipeline("Company Name", "Sender")
# → Research Agent → Analysis Agent → Personalization Agent
# → Complete personalized outreach strategy
```

### 🚀 **Real-World Applications**

This pattern directly applies to the HBR insights you've been studying:

**Sales Automation** (from your research):
- Research → Analysis → Personalization → Outreach
- Scales personalized outreach across hundreds of prospects

**Consulting Obelisk Model**:
- Research Synthesizer → Analysis → Client Briefing
- Replaces junior analyst work with AI orchestration

**B2B Omnichannel**:
- Customer Research → Journey Analysis → Multi-channel Personalization
- Orchestrates touchpoints across email, LinkedIn, phone, etc.

### �� **Next Steps for Learning**

**1. LangChain Comparison** (as planned):
- Rebuild the same workflow using LangChain
- Compare: code complexity, features, maintainability
- Understand what LangChain abstracts away

**2. Production Enhancements**:
- Add real API integrations (web scraping, CRM systems)
- Implement LLM integration for natural language generation
- Add A/B testing and performance optimization

**3. Advanced Orchestration**:
- Parallel agent execution
- Conditional workflows (if/then logic)
- Human approval workflows
- Multi-tenant scaling

### 💡 **Key Takeaways**

**For Your Learning Journey:**
- **Orchestration** is the superpower that makes AI agents valuable in business
- **State management** and **error handling** are critical for production systems
- **Clean architecture** makes systems maintainable and extensible
- **Monitoring and logging** provide visibility into complex workflows

**For Your Career:**
- You now understand the fundamentals of AI agent orchestration
- You can build production-ready multi-agent systems
- You have a foundation for learning LangChain and other frameworks
- You understand how to translate business processes into agent workflows

This is exactly the kind of **orchestration expertise** that the HBR articles identified as high-value! You're building the skills that will differentiate you as AI transforms knowledge work.
